Current and near-term quantum computers suffer from imperfections, as we repeatedly pointed it out. This is why we cannot run long algorithms, that is, deep circuits on them. A new breed of algorithms started to appear since 2013 that focus on getting an advantage from imperfect quantum computers. The basic idea is extremely simple: run a short sequence of gates where some gates are parametrized. Then read out the result, make adjustments to the parameters on a classical computer, and repeat the calculation with the new parameters on the quantum hardware. This way we create an iterative loop between the quantum and the classical processing units, creating classical-quantum hybrid algorithms.

<img src="../figures/hybrid_classical_quantum.svg" alt="Hybrid classical-quantum paradigm" style="width: 400px;"/>

These algorithms are also called variational to reflect the variational approach to changing the parameters. One of the most important example of this approach is the quantum approximate optimization algorithm, which is the subject of this notebook.

# Quantum approximate optimization algorithm

The quantum approximate optimization algorithm (QAOA) is a shallow-circuit variational algorithm for gate-model quantum computers that was inspired by quantum annealing. We discretize the adiabatic pathway in some $p$ steps, where $p$ influences precision. Each discrete time step $i$ has two parameters, $\beta_i, \gamma_i$. The classical variational algorithms does an optimization over these parameters based on the observed energy at the end of a run on the quantum hardware.

More formally, we want to discretize the time-dependent $H(t)=(1-t)H_0 + tH_1$ under adiabatic conditions. We achieve this by Trotterizing the unitary. For instance, for time step $t_0$, we can split this unitary as $U(t_0) = U(H_0, \beta_0)U(H_1, \gamma_0)$. We can continue doing this for subsequent time steps, eventually splitting up the evolution to $p$ such chunks:

$$
U = U(H_0, \beta_0)U(H_1, \gamma_0)\ldots U(H_0, \beta_p)U(H_1, \gamma_p).
$$

At the end of optimizing the parameters, this discretized evolution will approximate the adiabatic pathway:

<img src="../figures/qaoa_process.svg" alt="Quantum approximate optimization algorithm" style="width: 400px;"/>

The Hamiltonian $H_0$ is often referred to as the driving or mixing Hamiltonian, and $H_1$ as the cost Hamiltonian. The simplest mixing Hamiltonian is $H_0 = -\sum_i \sigma^X_i$, the same as the initial Hamiltonian in quantum annealing. By alternating between the two Hamiltonian, the mixing Hamiltonian drives the state towards an equal superposition, whereas the cost Hamiltonian tries to seek its own ground state.

Let us import the necessary packages first:

In [ ]:
import cirq
from cirq_tools import *
import numpy as np
from scipy.optimize import minimize
np.set_printoptions(precision=3, suppress=True)

 Now we can define our mixing Hamiltonian on some qubits. As in the notebook on classical and quantum many-body physics, we had to define, for instance, an `IZ` operator to express $\mathbb{I}\otimes\sigma_1^Z$, that is, the $\sigma_1^Z$ operator acting only on qubit 1. We can achieve the same effect in the following way (this time using the Pauli-X operator):

In Cirq, Pauli operators $\sigma^X$ are expressed with the objects `cirq.X`. Suppose that you have multiple qubits `q1`,`q2`,..., you can apply $\sigma^X$ to `q1` as `cirq.X(q1)`. This is equivalent to the `IX` operator.

# Mixing Hamiltonian
The next step is to define the unitary operator
$$
U(\beta,H_0) = e^{-i\pi\beta H_0/2},~~~ H_0 = \sum_{j=1}^n X_j,
$$
where $\beta$ is the variational parameter. Since the Pauli-$X$ operators on each qubit commute with each other, we can alternatively write this as
$$
U(\beta, H_0) = \prod_{j=1}^n e^{-i\pi\beta X_j/2}.
$$

So this is just a rotation of each qubit around the $X$-axis on the Bloch sphere by an amount determined by $\beta$. This operation is _not_ diagonal in the computational basis, and the resulting state will not be an equal superposition over all bitstrings. So after this step there will be constructive and destructive interference, which hopefully leads to enhancement of states corresponding to small values of $H_1$. Note that, up to an inconsequential global phase, we can also write
$$
U(\beta, H_0) = \prod_{j=1}^n X_j^{-\beta}.
$$

This dramatically simplifies the function to create a cirquit operator for $U(\beta, H_0)$. For example, if you have two qubits (a, b), the function `beta_layer` takes a variational parameter and qubits, and return the corresponding operator.

In [ ]:
def beta_layer(beta, a, b):
    yield (cirq.X**(-beta)).on_each([a, b])

You can use this function to create a circuit.

In [ ]:
a = cirq.NamedQubit("a")
b = cirq.NamedQubit("b")
circuit = cirq.Circuit()
circuit.append(beta_layer(0.1, a, b))
plot_circuit(circuit)

# Cost Hamiltonian
The next step is to define the unitary operator
$$
U(\gamma,H_1) = e^{-i\pi\gamma H_1/2}
$$

where $\gamma$ is the variational parameter. As an example, we will minimize the Ising problem defined by the cost Hamiltonian $H_1=-\sigma^Z_1 \otimes \sigma^Z_2$, whose minimum is reached whenever $\sigma^Z_1 = \sigma^Z_2$ (for the states $|-1, -1\rangle$, $|11\rangle$ or any superposition of both). In this case, by ajusting the definition of $\gamma$, we have
$$
U(\gamma, H_1) = e^{-i\pi\gamma Z_1Z_2}.
$$

We can create custom gates for unitary operators with Cirq. When making a custom gate, we can specify its action in a variety of ways. We'll illustrate it for the $ZZ$ gate:
$$
\begin{align}
\exp(-i \pi\gamma Z\otimes Z) = \begin{bmatrix}
e^{-i\pi\gamma} & 0 & 0 & 0 \\
0 & e^{i\pi\gamma} & 0 & 0 \\
0 & 0 & e^{i\pi\gamma} & 0 \\
0 & 0 & 0 & e^{-i\pi\gamma}
\end{bmatrix}
\end{align}
$$

One option is to specify a decomposition of your gate into gates that Cirq already knows about. First, the $ZZ$ gate can be decomosed into the four gates.
$$
\begin{align}
\exp(-i \pi\gamma Z\otimes Z) = 
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 &0 \\
0 & 0 & 0 & e^{-i\pi\gamma}
\end{bmatrix}
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & e^{i\pi\gamma} & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & e^{i\pi\gamma} & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
e^{-i\pi\gamma} & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
\end{align}
$$

For the first part, we can write it as a power of the $CZ$ gate (Controlled-$Z$):
$$
\begin{align}
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & e^{-i\pi\gamma}
\end{bmatrix}
= 
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & (e^{i\pi})^{-\gamma}
\end{bmatrix}
= CZ^{-\gamma}
\end{align}
$$

For the second part, we can move the phase around by conjugating by $X$ gates. For example,
$$
\begin{align}
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & e^{i\pi\gamma} & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
=
(I\otimes X)\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & e^{i\pi\gamma}
\end{bmatrix}
(I\otimes X)
=(I\otimes X)\cdot CZ^{\gamma}\cdot (I\otimes X)
\end{align}
$$

By applying the same technique, we have:
$$
\begin{align}
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & e^{-i\pi\gamma} & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
=(X\otimes I)\cdot CZ^{\gamma}\cdot (X\otimes I)
\end{align}
$$

$$
\begin{align}
\begin{bmatrix}
e^{i\pi\gamma} & 0 & 0 & 0\\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
=
(X\otimes X)\cdot CZ^{-\gamma}\cdot
(X\otimes X)
\end{align}
$$

By combining these results, we can decompose the $ZZ$ gate into the basic gates in Cirq.

$$
\begin{align}
\exp(-i \pi\gamma Z\otimes Z) = CZ^{-\gamma}\cdot
(I\otimes X)CZ^{\gamma}(I\otimes X)\cdot
(X\otimes I)CZ^{\gamma}(X\otimes I)\cdot
(X\otimes X)CZ^{-\gamma}(X\otimes X)
\end{align}
$$
$$
\begin{align}
= CZ^{-\gamma}
(I\otimes X)CZ^{\gamma}(X\otimes X)
CZ^{\gamma}(I\otimes X)CZ^{-\gamma}(X\otimes X)
\end{align}
$$

We can use this expression to define the custom $ZZ$ gate.

In [ ]:
class ZZGate(cirq.ops.gate_features.TwoQubitGate):
    def __init__(self, gamma):
        self.gamma = gamma
    
    def _decompose_(self, qubits):
        a, b = qubits
        yield cirq.CZ(a, b)**(-self.gamma),
        yield cirq.X(b),
        yield cirq.CZ(a, b)**self.gamma,
        yield cirq.X.on_each([a, b]),
        yield cirq.CZ(a, b)**self.gamma,
        yield cirq.X(b),
        yield cirq.CZ(a, b)**(-self.gamma)
        yield cirq.X.on_each([a, b]),

    # How should the gate look in ASCII diagrams?
    def _circuit_diagram_info_(self, args):
        return cirq.protocols.CircuitDiagramInfo(
            wire_symbols=('Z', 'Z'),
            exponent=self.gamma)

For example, if you have two qubits (a, b), the function `gamma_layer` takes a variational parameter and qubits, and return the corresponding operator.

In [ ]:
def gamma_layer(gamma, a, b):
    yield ZZGate(gamma).on(a, b)

circuit = cirq.Circuit()
circuit.append(gamma_layer(0.1, a, b))
circuit

We can check that the matrix expression of this operator is what we expeced.

In [ ]:
print (circuit.to_unitary_matrix().round(5))

gamma = 0.1
test_matrix = np.array([[np.exp(-1j*np.pi*gamma),0, 0, 0],
                        [0, np.exp(1j*np.pi*gamma), 0, 0],
                        [0, 0, np.exp(1j*np.pi*gamma), 0],
                        [0, 0, 0, np.exp(-1j*np.pi*gamma)]])
print ("\n")
print (test_matrix.round(5))

The function `create_cirquit` composes the unitary operators to create a quantum circuite to approximate the adiabatic pathway. The initial state is a uniform superposition of all the states. It can be created using Hadamard gates on all the qubits.

In [ ]:
def create_cirquit(betas, gammas):
    circuit = cirq.Circuit()
    circuit.append(cirq.H.on_each([a, b]))
    for beta, gamma in zip(betas, gammas):
        circuit.append(beta_layer(beta, a, b))
        circuit.append(gamma_layer(gamma, a, b))
    return circuit

For example, we can create a circuite with 4 steps with random parameters. 

In [ ]:
p = 4
betas = np.random.uniform(0, np.pi*2, p)
gammas = np.random.uniform(0, np.pi*2, p)
circuit = create_cirquit(betas, gammas)
circuit

To apply the optimizaion for variational parameters, we define two auxiliary functions. The function `energy_from_params` simulate the cirquit operation with given parameters, and return the expectation value of cost Hamiltonian for the resulting qubits state. `evaluate_circuit` is a wrapper function that the optimizer in SciPy can consume.

In [ ]:
def energy_from_params(betas, gammas):
    sim = cirq.Simulator()
    circuit = create_cirquit(betas, gammas)
    wf = sim.simulate(circuit).final_state
    return -np.sum(np.abs(wf)**2 * np.array([1, -1, -1, 1])) 

def evaluate_circuit(beta_gamma):
    betas = beta_gamma[:p]
    gammas = beta_gamma[p:]
    return energy_from_params(betas, gammas)

Finally, we optimize the angles:

In [ ]:
result = minimize(evaluate_circuit,
                  np.concatenate([betas, gammas]),
                  method='L-BFGS-B', options={'eps': 0.0001})
result

# Analysis of the results

We create a circuit using the optimal parameters found.

In [ ]:
betas = result['x'][:p]
gammas = result['x'][p:]
circuit = create_cirquit(betas, gammas)

We use the Cirq's simulator in order to display the state created by the circuit.

In [ ]:
wf = cirq.Simulator().simulate(circuit).final_state
wf

We see that the state is approximately $e^{i \theta} \frac{1}{\sqrt{2}} \left( |00 \rangle + |11 \rangle \right)$, where $\theta$ is a phase factor that doesn't change the probabilities. It corresponds to a uniform superposition of the two solutions of the classicial problem: $(\sigma_1=1$, $\sigma_2=1)$ and $(\sigma_1=-1$, $\sigma_2=-1)$

Let's now try to evaluate the operators $\sigma^Z_1$ and $\sigma^Z_2$ independently:

In [ ]:
print(np.sum(np.abs(wf)**2 * np.array([1, 1, -1, -1])))
print(np.sum(np.abs(wf)**2 * np.array([1, -1, 1, -1])))

We see that both are approximatively equal to zero. It's expected given the state we found above and corresponds a typical quantum behavior where $\mathbb{E}[\sigma^Z_1 \sigma^Z_2] \neq \mathbb{E}[\sigma^Z_1] \mathbb{E}[\sigma^Z_2]$